## Lab 6. Análisis de tráfico de red

### Miembros
* Fernanda Esquivel - 21542

### Link al repositorio
El repositorio puede ser visualizado [acá](https://github.com/FerEsq/SDS-Lab-06)

In [1]:
from scapy.all import rdpcap, IP, TCP, UDP
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

# Análisis estadístico

## Preprocesamiento de 'analisis_paquetes.pcap'

In [2]:
packages = rdpcap("analisis_paquetes.pcap")
print(f"   Se han cargado {len(packages)} paquetes del archivo PCAP\n")

   Se han cargado 62 paquetes del archivo PCAP



In [3]:
data = []

In [4]:
for pkt in packages:
    # Diccionario para almacenar la información del paquete actual
    pkt_info = {}
    
    # Verificamos si el paquete tiene una capa IP
    if IP in pkt:
        pkt_info['Src_Address'] = pkt[IP].src
        pkt_info['Dst_Address'] = pkt[IP].dst
        pkt_info['Protocol'] = pkt[IP].proto
        pkt_info['IP_Length'] = pkt[IP].len
        pkt_info['TTL'] = pkt[IP].ttl
        
        # Verificamos si el paquete tiene una capa TCP
        if TCP in pkt:
            pkt_info['Src_Port'] = pkt[TCP].sport
            pkt_info['Dst_Port'] = pkt[TCP].dport
            pkt_info['TCP_Flags'] = pkt[TCP].flags
            pkt_info['Seq_Num'] = pkt[TCP].seq
            pkt_info['Ack'] = pkt[TCP].ack
        # Verificamos si el paquete tiene una capa UDP
        elif UDP in pkt:
            pkt_info['Src_Port'] = pkt[UDP].sport
            pkt_info['Dst_Port'] = pkt[UDP].dport
        else:
            pkt_info['Src_Port'] = None
            pkt_info['Dst_Port'] = None
        
        # Agregamos la información del paquete a nuestra lista
        data.append(pkt_info)

In [5]:
#Creción del DataFrame con la info recopilada
df = pd.DataFrame(data)
print("DataFrame creado con éxito\n")

DataFrame creado con éxito



In [6]:
print("Contenido de las primeras 5 filas del dataset:")
print(df.head(5))
print("\n")

Contenido de las primeras 5 filas del dataset:
   Src_Address  Dst_Address  Protocol  IP_Length  TTL  Src_Port  Dst_Port
0   10.1.10.53  84.54.22.33        17        961   64        53        53
1  84.54.22.33   10.1.10.53        17         84  122        53        53
2   10.1.10.53  84.54.22.33        17        975   64        53        53
3  84.54.22.33   10.1.10.53        17         84  122        53        53
4   10.1.10.53  84.54.22.33        17       1012   64        53        53




In [7]:
print("Valores de las columnas principales:")
colsToShow = ['Src_Address', 'Dst_Address', 'Src_Port', 'Dst_Port']
print(df[colsToShow].head(10))  # Mostramos las primeras 10 filas
print("\n")

Valores de las columnas principales:
   Src_Address  Dst_Address  Src_Port  Dst_Port
0   10.1.10.53  84.54.22.33        53        53
1  84.54.22.33   10.1.10.53        53        53
2   10.1.10.53  84.54.22.33        53        53
3  84.54.22.33   10.1.10.53        53        53
4   10.1.10.53  84.54.22.33        53        53
5  84.54.22.33   10.1.10.53        53        53
6   10.1.10.53  84.54.22.33        53        53
7  84.54.22.33   10.1.10.53        53        53
8   10.1.10.53  84.54.22.33        53        53
9  84.54.22.33   10.1.10.53        53        53




## Estadísticas

### Todas las IP origen

In [18]:
#Mostrar todas las IP origen
print("   IPs origen únicas:")
ipsOrigin = df['Src_Address'].unique()
print(ipsOrigin)
print(f"      Total de IPs origen únicas: {len(ipsOrigin)}")
print("\n")

   IPs origen únicas:
['10.1.10.53' '84.54.22.33' '75.75.75.75']
      Total de IPs origen únicas: 3




### Todas las IP destino

In [9]:
#Mostrar todas las IP destino
print("   IPs destino únicas:")
ipsDestination = df['Dst_Address'].unique()
print(ipsDestination)
print(f"      Total de IPs destino únicas: {len(ipsDestination)}")
print("\n")

   IPs destino únicas:
['84.54.22.33' '10.1.10.53' '75.75.75.75']
      Total de IPs destino únicas: 3




### ¿Cuál es la IP origen más frecuente?

In [ ]:
#IP origen más frecuente
print("   IP origen más frecuente:")
ipOrFrequent = df['Src_Address'].value_counts().idxmax()
ipOrCount = df['Src_Address'].value_counts().max()
print(f"      IP: {ipOrFrequent} (aparece {ipOrCount} veces)")

   IP origen más frecuente:
      IP: 10.1.10.53 (aparece 31 veces)


#### ¿A qué IP destino se comunica con más frecuencia?

In [ ]:
print("      IP destino con la que se comunica más frecuentemente:")
# Filtramos el DataFrame para la IP origen más frecuente
dfIPFrequent = df[df['Src_Address'] == ipOrFrequent]
ipDestFrequent = dfIPFrequent['Dst_Address'].value_counts().idxmax()
ipDestCount = dfIPFrequent['Dst_Address'].value_counts().max()
print(f"         IP: {ipDestFrequent} (aparece {ipDestCount} veces)")

      IP destino con la que se comunica más frecuentemente:
         IP: 84.54.22.33 (aparece 29 veces)


#### ¿A qué puerto destino se comunica?

In [ ]:
print("      Puertos destino a los que se comunica:")
#Filtramos para la IP origen más frecuente y la IP destino más frecuente
dfIPPort = df[(df['Src_Address'] == ipOrFrequent) & (df['Dst_Address'] == ipDestFrequent)]
destinationPorts = dfIPPort['Dst_Port'].value_counts()
print(destinationPorts.head(10))  # Mostramos los 10 puertos más frecuentes

      Puertos destino a los que se comunica:
Dst_Port
53    29
Name: count, dtype: int64


##### ¿Cuál es el propósito de este puerto?

In [ ]:
#Identificamos el puerto más común
commonPort = destinationPorts.idxmax()
print(f"\n      Puerto destino más común: {commonPort}")


      Puerto destino más común: 53


In [15]:
#Diccionario con puertos comunes y sus propósitos
knownPorts = {
    20: "FTP (transferencia de datos)",
    21: "FTP (control)",
    22: "SSH (acceso remoto seguro)",
    23: "Telnet (acceso remoto)",
    25: "SMTP (correo saliente)",
    53: "DNS (resolución de nombres)",
    80: "HTTP (web)",
    110: "POP3 (correo entrante)",
    143: "IMAP (correo entrante)",
    443: "HTTPS (web segura)",
    3389: "RDP (escritorio remoto)",
    8080: "HTTP Alternativo/Proxy"
}

In [16]:
# Verificamos si el puerto está en nuestro diccionario
if commonPort in knownPorts:
    print(f"      Propósito: {knownPorts[commonPort]}")
else:
    #Si no está en nuestro diccionario, comprobamos algunos rangos comunes
    if 0 <= commonPort <= 1023:
        print("      Propósito: Puerto bien conocido (sistema)")
    elif 1024 <= commonPort <= 49151:
        print("      Propósito: Puerto registrado (aplicaciones)")
    else:
        print("      Propósito: Puerto dinámico/privado (uso temporal)")

      Propósito: DNS (resolución de nombres)


#### ¿Desde que puertos origen se comunica?

In [ ]:
print("      Puertos origen desde los que se comunica:")
originPorts = dfIPPort['Src_Port'].value_counts()
print(originPorts.head(10))  #Mostramos los 10 puertos origen más frecuentes

      Puertos origen desde los que se comunica:
Src_Port
53    29
Name: count, dtype: int64
